# **Hercoderen gemeenten 2023**

In 2023 worder er een drie gemeenten samengevoegd tot een nieuwe gemeent. Dit werkt door op een aantal plaatsen in de database:

1. In de definitie van de PROTOS, er moeten voor de nieuwe gemeente Protos worden aangemaakt en voor de oude gemeenten niet meer.
2. In de data voor de Rioolwaterzuiveringsinstallaties (RWZI's). Waarschijnlijk wordt dit gewijzigd aangeleverd door RIVM, maar voor de zekerheid is er code ontwikkeld om dit op te vangen mocht de data in de oude indeling worden aangeleverd.
3. In de Static tabellen met bevolkingsaantallen. Dit start met de tabel Population\_Base en daarvan worden andere Static-tabellen afgeleid. In de Stored Procedure die de data van Stage naar Inter kopieert, wordt de nieuwe gemeentelijke indeling aangebracht m.b.v. een mapping-tabel. Alle tabellen die afhankelijk zijn van VWSInter.Population\_Base zullen nu ook de nieuwe gemeentelijke indeling krijgen. 
4. In één tabel wordt bevolkingsgegevens per gemeente ingelezen in het VWSStage-schema. De stored-procedure die de data kopieert naar VWSStatic worden aangepast om de data in de nieuwe indeling in VWSStatic te schrijven op dezelfde manier als dat voor de Population\_Base tabel gebeurt. 
5. Een tabel met gemeentelijke bevolkingsaantallen wordt direct ingelezen in het VWSStatic schema. Dit wordt aangepast door deze eerst in het VWSStage schema in te lezen en dan middels een Stored Procedure de gegevens in de juiste indeling in het VWSStatic schema te schrijven (zelfde werkwijze als bij 3 en 4). 
6. In één van de views wordt gerefereerd aan de tabel Safety\_Regions\_per\_Municipal. Hierin staat de relatie tussen gemeenten en veiligheidsregios. Toegevoegd wordt dat alleen de meest recente dataset met deze relatie wordt gebruikt, zodat na het inlezen van een dataset met de nieuwe gemeentelijke indeling, deze view ook direct goed staat.
7. In de tabel VWSSTATIC.Active\_Municipalities staat een lijst met alle actieve gemeenten. Hier moet een nieuwe set aan worden toegevoegd waarin de drie 'oude' gemeenten niet meer voorkomen en waaraan Voorne aan Zee is toegevoegd als nieuwe gemeente. 

> N.B. voor 6 moet dan wel een nieuw bestand met Safety\_Regions\_per\_Municipal worden klaargezet in ingelezen. 

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb",
        "src/utils/protos.ipynb",
        "src/statics/safety_regions_per_municipality.ipynb",
        "src/statics/safety_regions/inhabitants_per_safety_region.ipynb",
        "src/dataflows/rivm_number_municipality.ipynb"
    ]
}
```

## **1    Aanpassen Protos**

In [ ]:
/*
	Aanpassen lijst met gemeenten waarvoor een Proto-bestand moet worden aangemaakt, 
	naar aanleiding van de gemeentelijke herindeling. 

	Onderin staat uit-gecomment de sources om dit terug te draaien. 
*/

DECLARE @GemCode nvarchar(50)
DECLARE @strSQL nvarchar(255)

-- Toevoegen Voorne aan Zee: GM1992
SET @GemCode = 'GM1992'
SET @strSQL = 'DELETE FROM DataTino_Proto_1.Protos WHERE Name = ' + Char(39) + @GemCode + Char(39)
Exec(@strSQL)

SET @strSQL = 'INSERT INTO DataTino_Proto_1.Protos (Header_Names, Header_Values, Active, [Name], [Description]) 
	VALUES (''name|code|proto_name|last_generated'',' + Char(39) + @GemCode + '|' +@GemCode + '|' + @GemCode + '|=GETDATE()' + Char(39) + ',1,' + Char(39) + @GemCode + Char(39) + ', ' + Char(39) + 'Proto File for ' +  @GemCode + Char(39) + ')'
Exec(@strSQL)

-- DeActiveren Brielle (GM0501), Hellevoetsluis (GM0530), Westvoorne (GM0614)
UPDATE A SET ACTIVE = 0 FROM DataTino_Proto_1.Protos A WHERE [Name] IN ('GM0501','GM0530','GM0614')


/*
	Terugdraaien wijzigingen in Protos
	
	UPDATE A SET ACTIVE = 1 FROM DataTino_Proto_1.Protos A WHERE [Name] IN ('GM0501','GM0530','GM0614')

	UPDATE A SET ACTIVE = 0 FROM DataTino_Proto_1.Protos A WHERE [Name] IN ('GM1992')

*/


## **2    Aanpassingen voor RWZI's**

Hier wordt gebruik gemaakt van de al aanwezige stored procedure **SP\_MERGE\_MOVE\_STATIC\_CBS\_RWZI** om wijzigingen aan te brengen in de mapping van RWZI's naar gemeenten. Hiervoor wordt er een aantal regels toegevoegd aan het csv-bestand "**Aantal\_inwoners\_per\_verzorgingsgebied\_mutaties**.csv" in de blob-storage, Datafiles\\Static map. 

Inhoud van deze csv wordt dan:

rwzi\_code;rwzi\_naam;startdatum;einddatum;inwoners;regio\_type;regio\_code;regio\_naam;aandeel;toelichting

9043;LIENDEN;01-01-2020;07-12-2020;6728;GM;GM1740;Neder-Betuwe;1.5%;definitief

9043;LIENDEN;07-12-2020;31-12-2020;0;GM;GM1740;Neder-Betuwe;0.0%;definitief

9043;LIENDEN;01-01-2020;07-12-2020;6728;GM;GM0214;Buren;98.5%;definitief

9043;LIENDEN;07-12-2020;31-12-2020;0;GM;GM0214;Buren;0.0%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM0214;Buren;5.2%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM0214;Buren;15.3%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM0281;Tiel;75.3%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM0281;Tiel;66.7%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM1740;Neder-Betuwe;13.7%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM1740;Neder-Betuwe;12.7%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM1960;West Betuwe;5.9%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM1960;West Betuwe;5.3%;definitief

9029;AALST;01-01-2020;05-10-2020;10495;GM;GM0297;Zaltbommel;97.8%;definitief

9029;AALST;05-10-2020;31-12-2020;0;GM;GM0297;Zaltbommel;0.0%;definitief

9029;AALST;01-01-2020;05-10-2020;10495;GM;GM0263;Maasdriel;2.2%;definitief

9029;AALST;05-10-2020;31-12-2020;0;GM;GM0263;Maasdriel;0.0%;definitief

9041;ZALTBOMMEL;01-01-2020;05-10-2020;41974;GM;GM0263;Maasdriel;55.8%;definitief

9041;ZALTBOMMEL;05-10-2020;31-12-2020;53462;GM;GM0263;Maasdriel;45.1%;definitief

9041;ZALTBOMMEL;01-01-2020;05-10-2020;41974;GM;GM0297;Zaltbommel;44.2%;definitief

9041;ZALTBOMMEL;05-10-2020;31-12-2020;53462;GM;GM0297;Zaltbommel;54.9%;definitief

12014;BEEMSTER;01-01-2020;31-12-9999;104382;GM;GM0370;Beemster;0.0%;definitief

12041;BEEMSTER;01-01-2020;31-12-9999;104382;GM;GM0439;Purmerend;9.7%;definitief

12013;GEESTMERAMBACHT;01-01-2020;31-12-9999;185792;GM;GM0398;Heerhugowaard;0%;definitief

12013;GEESTMERAMBACHT;01-01-2020;31-12-9999;185792;GM;GM0416;Langedijk;0%;definitief

12013;GEESTMERAMBACHT;01-01-2020;31-12-9999;185792;GM;GM1980;Dijk en Waard;46.7%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM0786;Grave;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0756;Boxmeer;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0786;Grave;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0815;Mill en Sint Hubert;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1684;Cuijk;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1702;Sint Anthonis;0%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM1982;Land van Cuijk;6.91%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1982;Land van Cuijk;90.07%;definitief

28005;DINTHER;01-01-2020;31-12-9999;123719;GM;GM0856;Uden;0%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM1685;Landerd;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0856;Uden;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1685;Landerd;0%;definitief

28005;DINTHER;01-01-2020;31-12-9999;123719;GM;GM1991;Maashorst;32.60%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM1991;Maashorst;5.64%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1991;Maashorst;9.91%;definitief

## **3    Aanpassen bevolkingsaantallen Population\_Base**

Om de gemeentelijke indeling te kunnen aanpassen naar de gewenste indeling wordt er een mapping tabel gemaakt die de wijzigingen bevat. Met behulp van deze mapping-tabel wordt de data die van VWSTAGE naar VWSSTATIC wordt gekopieerd, direct in de goede gemeentelijke indeling gezet. Dit gebeurt in de Stored Procedure **SP\_CBS\_POPULATION\_BASE\_STATIC**.

N.B. de kolom "Active" in de tabel RECLASSIFIED_MUNICIPALITY_MAPPING staat default op 0, wat betekent dat deze grenswijzigingen _NIET_ worden uitgevoerd. Om ze actief te maken moet met een update commando de waarde in de kolom "Active" op 1 worden gezet: 

Update VWSStatic.RRECLASSIFIED_MUNICIPALITY_MAPPING SET Active = 1

FROM VWSStatic.RECLASSIFIED_MUNICIPALITY_MAPPING 

WHERE Date\_Last\_Inserted = (SELECT Max(Date\_Last\_Inserted) )

In [ ]:
/*
	Hier wordt een tabel gemaakt waarin informatie wordt
	opgeslagen omtrent herindelingen van gemeenten. 
*/

DROP TABLE IF EXISTS VWSSTATIC.RRECLASSIFIED_MUNICIPALITY_MAPPING

CREATE TABLE VWSSTATIC.RECLASSIFIED_MUNICIPALITY_MAPPING
(
	[ID] [int] IDENTITY(1,1) NOT NULL,
	[DATE_LAST_INSERTED] [datetime] NOT NULL Default GetDate(),
	[GM_CODE_ORIGINAL] [nvarchar](10) NULL,
	[GM_NAME_ORIGINAL] [nvarchar](255) NULL,
	[GM_CODE_NEW] [nvarchar](10) NULL,
	[GM_NAME_NEW] [nvarchar](255) NULL,
	[GM_NAME_NEW] [nvarchar](10) NULL,
	[VR_NAME] [nvarchar](255) NULL,
	[PROVINCE_CODE] [nvarchar](10) NULL,
	[PROVINCE_NAME] [nvarchar](255) NULL,
	[GGD_CODE] [nvarchar](10) NULL,
	[GGD_NAME] [nvarchar](255) NULL,
	[SHARES] [decimal](16, 6) NOT NULL,
	[ACTIVE] [int] NOT NULL Default(0)
)

INSERT INTO VWSSTATIC.RECLASSIFIED_MUNICIPALITY_MAPPING (GM_CODE_ORIGINAL,	[GM_NAME_ORIGINAL],GM_CODE_NEW,GM_NAME_NEW,VR_CODE, VR_NAME,PROVINCE_CODE,PROVINCE_NAME,GGD_CODE,GGD_NAME,SHARES,ACTIVE)
VALUES ('GM1684','Cuijk','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
     , ('GM0756','Boxmeer','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	 , ('GM0815','Mill en Sint Hubert','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	 , ('GM1702','Sint Anthonis','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	 , ('GM0786','Grave','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	 , ('GM1685','Landerd','GM1991','Maashorst','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	 , ('GM0856','Uden','GM1991','Maashorst','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	 , ('GM0398','Heerhugowaard','GM1980','Dijk en Waard','VR10','Noord-Holland-Noord','PV27','Noord-Holland','GG2707','GGD Hollands-Noorden',1,0)
	 , ('GM0416','Langedijk','GM1980','Dijk en Waard','VR10','Noord-Holland-Noord','PV27','Noord-Holland','GG2707','GGD Hollands-Noorden',1,0)
	 , ('GM0370','Beemster','GM0439','Purmerend','VR11','Zaanstreek-Waterland','PV27','Noord-Holland','GG7306','GGD Zaanstreek/Waterland',1,0)
	 , ('GM0501','Brielle','GM1992','Voorne aan Zee','VR17','Rotterdam-Rijnmond','PV28','Zuid-Holland','GG4607','GGD Rotterdam-Rijnmond',1,0)
	 , ('GM0530','Hellevoetsluis','GM1992','Voorne aan Zee','VR17','Rotterdam-Rijnmond','PV28','Zuid-Holland','GG4607','GGD Rotterdam-Rijnmond',1,0)
	 , ('GM0614','Westvoorne','GM1992','Voorne aan Zee','VR17','Rotterdam-Rijnmond','PV28','Zuid-Holland','GG4607','GGD Rotterdam-Rijnmond',1,0)
GO



## **Aanpassen Stored Procedure SP\_CBS\_POPULATION\_BASE\_STATIC**

In [ ]:
CREATE OR ALTER PROCEDURE [dbo].[SP_CBS_POPULATION_BASE_STATIC]
AS
BEGIN
/*
	POPULATION_BASE is transferred from STAGE to STATIC,
	the table RECLASSIFIED_MUNICIPALITY_MAPPING is used to apply changes
	in the municipalities.
	
	The RECLASSIFIED_MUNICIPALITY_MAPPING table is left-joined, so only
	municipalities that met with a change will be affected.
	Only changes that are 'active' are selected.
*/
 -- POPULATION_BASE IS TRANSFERRED FROM STAGE TO STATIC, THE TABLE RECODE_MUNICIPALITY IS USED TO APPLY CHANGES IN THE MUNICIPALITIES.
 -- THE RECODE_MUNICIPALITY TABLE IS LEFT-JOINED, SO ONLY MUNICIPALITIES THAT MET WITH A CHANGE WILL BE AFFECTED. 
 -- ONLY CHANGES THAT ARE 'ACTIVE' ARE SELECTED.
 INSERT INTO [VWSSTATIC].[CBS_POPULATION_BASE] (
     [GEMEENTE_CODE]
     ,[GEMEENTE]
     ,[LEEFTIJD]
     ,[GESLACHT]
     ,[DATUM_PEILING]
     ,[POPULATIE]
     ,[VEILIGHEIDSREGIO_CODE]
     ,[VEILIGHEIDSREGIO_NAAM]
     ,[PROVINCIE_CODE]
     ,[PROVINCIE_NAAM]
     ,[GGD_CODE]
     ,[GGD_NAAM]
 )
 SELECT 
     ISNULL(B.[GM_CODE_NEW],[GEMEENTE_CODE]) AS [GEMEENTE_CODE]
     ,ISNULL(B.[GM_NAME_NEW],[GEMEENTE]) AS [GEMEENTE]
     ,A.[LEEFTIJD]
     ,A.[GESLACHT]
     ,CAST(A.[DATUM_PEILING] AS DATE) AS [DATUM_PEILING]
     ,ROUND(SUM(CAST(A.[POPULATIE] AS INT) * ISNULL(B.[SHARES],1.0)),0) AS [POPULATIE] 
     ,ISNULL(B.[VR_CODE],[VEILIGHEIDSREGIO_CODE]) AS [VEILIGHEIDSREGIO_CODE]
     ,ISNULL(B.[VR_NAME],[VEILIGHEIDSREGIO_NAAM]) AS [VEILIGHEIDSREGIO_NAAM]
     ,ISNULL(B.[PROVINCE_CODE],[PROVINCIE_CODE]) AS [PROVINCIE_CODE]
     ,ISNULL(B.[PROVINCE_NAME],[PROVINCIE_NAAM]) AS [PROVINCIE_NAAM]
     ,ISNULL(B.[GGD_CODE], A.[GGD_CODE]) AS [GGD_CODE]
     ,ISNULL(B.[GGD_NAME],[GGD_NAAM]) AS [GGD_NAAM]
 FROM [VWSSTAGE].[CBS_POPULATION_BASE] A
     LEFT JOIN (
         SELECT 
             * 
         FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
         WHERE [ACTIVE] = 1 
             AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
     ) B ON A.[GEMEENTE_CODE] = B.[GM_CODE_ORIGINAL]
 WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[CBS_POPULATION_BASE])
 GROUP BY
     ISNULL(B.[GM_CODE_NEW],[GEMEENTE_CODE]) 
     ,ISNULL(B.[GM_NAME_NEW],[GEMEENTE]) 
     ,A.[LEEFTIJD]
     ,A.[GESLACHT]
     ,CAST(A.[DATUM_PEILING] AS DATE) 
     ,ISNULL(B.[VR_CODE],[VEILIGHEIDSREGIO_CODE]) 
     ,ISNULL(B.[VR_NAME],[VEILIGHEIDSREGIO_NAAM]) 
     ,ISNULL(B.[PROVINCE_CODE],[PROVINCIE_CODE]) 
     ,ISNULL(B.[PROVINCE_NAME],[PROVINCIE_NAAM]) 
     ,ISNULL(B.[GGD_CODE],A.[GGD_CODE]) 
     ,ISNULL(B.[GGD_NAME],[GGD_NAAM]) 
 ORDER BY [DATUM_PEILING], ISNULL(B.[GM_CODE_NEW],[GEMEENTE_CODE])
 END
 
GO

## **4    Aanpassen Stored Procedure voor** **CBS\_INHABITANTS\_PER\_MUNICIPALITY**

De SP die de data van VWSStage naar VWSStatic kopieert wordt uitgebreid met een join naar de tabel RECLASSIFIED_MUNICIPALITY_MAPPING om de gegevens direct om te zetten naar de nieuwe gemeentelijke indeling.

In [ ]:
CREATE OR ALTER PROCEDURE [dbo].[SP_INHABITANTS_PER_REGION_STATIC]
AS
BEGIN
     INSERT INTO [VWSSTATIC].[CBS_INHABITANTS_PER_MUNICIPALITY]  (
         [VRCODE], 
         [GMCODE], 
         [INHABITANTS]
     )
     SELECT
         ISNULL(B.[VR_CODE], [CODE_42]),
         ISNULL(B.[GM_CODE_NEW], [CODE_1]),
         CAST(ROUND(SUM(CAST(A.[INWONERTAL_50] AS INT) * ISNULL(B.[SHARES],1.0)),0) AS INT)
     FROM [VWSSTAGE].[CBS_INHABITANTS_PER_MUNICIPALITY] A
         LEFT JOIN (
                     SELECT 
                         * 
                     FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
                     WHERE [ACTIVE] = 1 
                         AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
                 ) B ON A.[CODE_1] = B.[GM_CODE_ORIGINAL]
     WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSSTAGE].[CBS_INHABITANTS_PER_MUNICIPALITY])
     GROUP BY ISNULL(B.[VR_CODE],[CODE_42]), ISNULL(B.[GM_CODE_NEW],[CODE_1])
 END
GO

## **5    Toevoegen tabel in VWSStage en Stored Procedure naar VWSInter voor tabel.**

De tabel INHABITANTS\_PER\_MUNICIPALITY wordt direct in het VWSStatic schema ingelezen. Hiervoor wordt een tabel aangemaakt in het VWSStage schema alsemede een stored procedure om de data van Stage naar Static te kopieren en gedurende de kopieer-actie aan te passen aan de gewenste gemeentelijke indeling. 

Daarnaast moet er nog een Process worden toegevoegd aan de workflow.

In [ ]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

DROP TABLE IF EXISTS VWSSTAGE.INHABITANTS_PER_MUNICIPALITY

CREATE TABLE [VWSSTAGE].[INHABITANTS_PER_MUNICIPALITY](
	[REGIO] [varchar](50) NOT NULL,
	[GMCODE] [varchar](50) NOT NULL,
	[VRCODE] [varchar](50) NOT NULL,
	[VRNAME] [varchar](50) NOT NULL,
	[INHABITANTS] [varchar] (50) NOT NULL,
	[DATE_LAST_INSERTED] [datetime] NULL
) ON [PRIMARY]
GO
SET ANSI_PADDING ON
GO

In [ ]:
CREATE OR ALTER PROCEDURE dbo.SP_INHABITANTS_PER_MUNICIPALITY_STAGE_TO_STATIC
AS
BEGIN

      INSERT INTO [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY] (
             [REGIO], 
             [GMCODE], 
             [VRCODE], 
             [VRNAME], 
             [INHABITANTS], 
             [DATE_LAST_INSERTED]
         )
     SELECT  ISNULL(B.[GM_NAME_NEW],A.[REGION]),
             ISNULL(B.[GM_CODE_NEW],A.[GM_CODE]),
             ISNULL(B.[VR_CODE],A.[VR_CODE]),
             ISNULL(B.[VR_NAME],A.[VR_NAME]),
             CAST(ROUND(SUM(CAST(A.[INHABITANTS] AS INT) * ISNULL(B.[SHARES],1.0)),0) AS INT),
             MAX(A.[DATE_LAST_INSERTED])
     FROM [VWSSTAGE].[INHABITANTS_PER_MUNICIPALITY] A
     LEFT JOIN (
         SELECT 
             * 
         FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
         WHERE [ACTIVE]=1
             AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
     ) B ON A.[GM_CODE] = B.[GM_CODE_ORIGINAL]
     WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[INHABITANTS_PER_MUNICIPALITY])    
     GROUP BY ISNULL(B.[GM_NAME_NEW],A.[REGION]),
             ISNULL(B.[GM_CODE_NEW],A.[GM_CODE]),
             ISNULL(B.[VR_CODE],A.[VR_CODE]),
             ISNULL(B.[VR_NAME],A.[VR_NAME])
 END
 
GO

## **6    Aanpassen view die gebruik maakt van VWSStatic.Safety\_Regions\_per\_Municipal**

Hieronder wordt een nieuwe dataset weggeschreven in de tabel VWSSTATIC.Safety\_Regions\_per\_Municipal waarin de nieuwe gemeente Voorne aan Zee is opgenomen.

In de view VWSReport.V\_Aandeel\_Positief\_Getest\_per\_Regio wordt gebruik gemaakt van de tabel VWSStatic.Safety\_Regions\_per\_Municipal. Hier staat de relatie in tussen gemeenten en veiligheidsregios. Om te verzekeren dat de meest recente dataset wordt gebruikt, is in de subquery waar deze tabel wordt gebruikt, toegevoegd dat de meest recente dataset gebruikt moet worden.

In [ ]:
-- Hier wordt een nieuwe set toegevoegd aan de tabel VWSSTATIC.Safety_Regions_per_Municipal door alle gemeenten te nemen
-- die NIET zijn gegrenswijzigd en daar aan toe te voegen de nieuwe gemeente Voorne aan Zee

-- Variabele met DateTime, zodat alle regels dezelfde DateTime krijgen
DECLARE @DateTime as DateTime = GetDate()

INSERT INTO VWSSTATIC.SAFETY_REGIONS_PER_MUNICIPAL (GMCODE, GMNAAM, VRCODE, VRNAAM, DATE_LAST_INSERTED)
SELECT [GMCODE]
      ,[GMNAAM]
      ,[VRCODE]
      ,[VRNAAM]
      ,@DateTime
  FROM [VWSSTATIC].[SAFETY_REGIONS_PER_MUNICIPAL]
  WHERE DATE_LAST_INSERTED = (Select Max(DATE_LAST_INSERTED) FROM VWSSTATIC.SAFETY_REGIONS_PER_MUNICIPAL)
    AND NOT GMCODE IN ('GM0501','GM0530','GM0614','GM1992')

INSERT INTO VWSSTATIC.SAFETY_REGIONS_PER_MUNICIPAL (GMCODE, GMNAAM, VRCODE, VRNAAM, DATE_LAST_INSERTED)
VALUES ('GM1992','Voorne aan Zee','VR17','Rotterdam-Rijnmond',@DateTime)
GO

In [ ]:
/*
	Deze procedure maakt gebruik van de tabel Safety_Regions_per_municipal
	voor de relatie tussen gemeenten en veiligheidsregios. 
*/

CREATE OR ALTER VIEW [VWSREPORT].[V_AANDEEL_POSITIEF_GETEST_PER_REGIO] AS
-- Select base records
WITH BASE_CTE AS (
SELECT 
	DATE_OF_PUBLICATION AS DATE_OF_REPORT
,	SECURITY_REGION_CODE AS VRCODE
,	MUNICIPALITY_CODE AS GMCODE
,	TOTAL_REPORTED
,	INHABITANTS
,   T1.DATE_LAST_INSERTED
FROM VWSINTER.RIVM_COVID_19_NUMBER_MUNICIPALITY T1
LEFT JOIN [VWSSTATIC].[INHABITANTS_PER_SAFETY_REGION]   T3	ON SECURITY_REGION_CODE	= T3.VGRNR
WHERE T1.DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSINTER.RIVM_COVID_19_NUMBER_MUNICIPALITY)
)

,SECOND_CTE AS (
SELECT
    DATE_OF_REPORT
,   VRCODE
,   SUM(TOTAL_REPORTED) AS [TOTAL_REPORTED]
,   SUM(CAST(TOTAL_REPORTED AS FLOAT))/(CAST(INHABITANTS AS decimal(10,2))/CAST(100000 AS decimal(10,2))) AS [REPORTED_PER_REGION_100000]
,   INHABITANTS
,   DATE_LAST_INSERTED
FROM BASE_CTE
GROUP BY DATE_LAST_INSERTED, DATE_OF_REPORT, VRCODE, INHABITANTS
)

-- Final select and insert into statement. This part adds the calculation of a moving average of increased hospital admissions and cumulative calculations. It calculates it based on the previous 2 days as can be seen in row 59, after BETWEEN.

,THIRD_CTE AS  (
	SELECT
		T_SECOND.DATE_OF_REPORT
	,	T_SECOND.VRCODE
	,   ROUND((SUM(TOTAL_REPORTED) OVER (PARTITION BY T_SECOND.DATE_LAST_INSERTED, T_SECOND.VRCODE ORDER BY T_SECOND.DATE_OF_REPORT ASC)) / (CAST(INHABITANTS AS FLOAT)/100000.0), 2) AS DAGTOTAAL
	,	ROUND(REPORTED_PER_REGION_100000, 1) AS [TOENAME_TOV_T-1]
	FROM SECOND_CTE AS T_SECOND
	where T_SECOND.date_of_report between ((SELECT MAX(DATE_OF_REPORT) FROM SECOND_CTE) - DAY(6)) and (SELECT MAX(DATE_OF_REPORT) FROM SECOND_CTE)
	and vrcode != ''
)

,FOURTH_CTE AS (
	select DATE_OF_REPORT
		, SUM(DAGTOTAAL) TOTAAL_WEEK
	FROM
	(
		select  distinct a.*
		from THIRD_CTE a
		where date_of_report = (select max(DATE_OF_REPORT) from THIRD_CTE)
	) a
	group by DATE_OF_REPORT
)

select a.DATE_OF_REPORT
, VRCODE
, VRNAAM
, (CAST(DAGTOTAAL AS FLOAT)/b.TOTAAL_WEEK) * 100 [AANDEEL]
, DAGTOTAAL
, [TOENAME_TOV_T-1]
FROM(
select  distinct a.*, b.VRNAAM
from THIRD_CTE a
	join (
			SELECT * 
			FROM [VWSSTATIC].[SAFETY_REGIONS_PER_MUNICIPAL] 
			WHERE DATE_LAST_INSERTED=(SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.SAFETY_REGIONS_PER_MUNICIPAL)
		) B	on a.VRCODE = b.VRCODE
	where date_of_report = (select max(DATE_OF_REPORT) from THIRD_CTE)
) a, FOURTH_CTE b
GO

## **7    Aanpassen tabel met overzicht actieve gemeenten**

In de tabel <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">[VWSSTATIC].[ACTIVE_MUNICIPALITIES]</span> staan alle gemeenten die actief zijn in de actuele indeling per 1-1 van het lopende jaar. Aan deze tabel moeten worden toegevoegd: alle gemeenten die niet zijn gegrenswijzigd en de nieuwe gemeente Voorne aan Zee.

In [ ]:
IF (NOT EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'ACTIVE_MUNICIPALITIES'))
BEGIN
      CREATE TABLE [VWSSTATIC].[ACTIVE_MUNICIPALITIES](
            [GEMEENTE_CODE] [varchar](10) NULL,
            [GEMEENTE] [varchar](100) NULL,
            [VEILIGHEIDSREGIO_CODE] [varchar](10) NULL,
            [VEILIGHEIDSREGIO_NAAM] [varchar](100) NULL,
            [PROVINCIE_CODE] [varchar](10) NULL,
            [PROVINCIE_NAAM] [varchar](100) NULL,
            [GGD_CODE] [varchar](10) NULL,
            [GGD_NAAM] [varchar](100) NULL,
            [DATE_LAST_INSERTED] [datetime] NULL
      )     
END
GO

In [ ]:
-- Variabele met tijdstip van inserten, zodat alle gemeenten zelfde DateTime krijgen
DECLARE @DateTime as DateTime = GetDate()

-- Alle gemeenten toevoegen die niet zijn gewijzigd.
INSERT INTO [VWSSTATIC].[ACTIVE_MUNICIPALITIES] (
       [GEMEENTE_CODE]
      ,[GEMEENTE]
      ,[VEILIGHEIDSREGIO_CODE]
      ,[VEILIGHEIDSREGIO_NAAM]
      ,[PROVINCIE_CODE]
      ,[PROVINCIE_NAAM]
      ,[GGD_CODE]
      ,[GGD_NAAM]
      ,Date_Last_Inserted)
SELECT [GEMEENTE_CODE]
      ,[GEMEENTE]
      ,[VEILIGHEIDSREGIO_CODE]
      ,[VEILIGHEIDSREGIO_NAAM]
      ,[PROVINCIE_CODE]
      ,[PROVINCIE_NAAM]
      ,[GGD_CODE]
      ,[GGD_NAAM]
      ,@DateTime
FROM VWSSTATIC.ACTIVE_MUNICIPALITIES
WHERE Date_Last_Inserted = (Select Max(Date_Last_Inserted) FROM VWSSTATIC.ACTIVE_MUNICIPALITIES)
    AND NOT GEMEENTE_CODE IN ('GM0501','GM0530','GM0614','GM1992')

-- Nieuwe gemeente Voorne aan Zee toevoegen 
INSERT INTO [VWSSTATIC].[ACTIVE_MUNICIPALITIES] (
       [GEMEENTE_CODE]
      ,[GEMEENTE]
      ,[VEILIGHEIDSREGIO_CODE]
      ,[VEILIGHEIDSREGIO_NAAM]
      ,[PROVINCIE_CODE]
      ,[PROVINCIE_NAAM]
      ,[GGD_CODE]
      ,[GGD_NAAM]
      ,Date_Last_Inserted)
VALUES ('GM1992','Voorne aan Zee','VR17','Rotterdam-Rijnmond','PV28','Zuid-Holland','GG4607','GGD Rotterdam-Rijnmond',@DateTime)
GO